# Test Deployed AutoML Models

This notebook demonstrates how to:
1. List and verify trained models
2. List and verify model endpoints
3. Make predictions using both vision and tabular models

In [ ]:
import os
from google.cloud import aiplatform
from google.cloud import storage
import pandas as pd
from PIL import Image
import io

# Configuration
PROJECT_ID = "386465169172"
REGION = "us-central1"
BUCKET_NAME = "qwiklabs-gcp-00-ffe6db11d36b-bucket"

# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

## 1. List and Verify Trained Models

In [ ]:
def list_models():
    """List all models in the project"""
    models = aiplatform.Model.list()
    print("Available Models:")
    for model in models:
        print(f"\nModel Name: {model.display_name}")
        print(f"Resource Name: {model.resource_name}")
        print(f"Create Time: {model.create_time}")
        
list_models()

## 2. List and Verify Model Endpoints

In [ ]:
def list_endpoints():
    """List all model endpoints in the project"""
    endpoints = aiplatform.Endpoint.list()
    print("Available Endpoints:")
    for endpoint in endpoints:
        print(f"\nEndpoint Name: {endpoint.display_name}")
        print(f"Resource Name: {endpoint.resource_name}")
        print(f"Create Time: {endpoint.create_time}")
        
list_endpoints()

## 3. Test Vision Model Predictions

In [ ]:
def test_vision_model(endpoint_id, image_uri):
    """Test vision model with a sample image
    
    Args:
        endpoint_id: The endpoint resource name
        image_uri: GCS URI of the test image
    """
    endpoint = aiplatform.Endpoint(endpoint_id)
    
    # Download image from GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(image_uri.split(f'gs://{BUCKET_NAME}/')[-1])
    image_bytes = blob.download_as_bytes()
    
    # Make prediction
    prediction = endpoint.predict([{"image_bytes": image_bytes}])
    print("Vision Model Prediction:")
    print(prediction)

# Replace with your vision endpoint ID and test image URI
vision_endpoint_id = "YOUR_VISION_ENDPOINT_ID"
test_image_uri = f"gs://{BUCKET_NAME}/data/test_image.jpg"

# Uncomment to test
# test_vision_model(vision_endpoint_id, test_image_uri)

## 4. Test Tabular Model Predictions

In [ ]:
def test_tabular_model(endpoint_id):
    """Test tabular model with sample data"""
    endpoint = aiplatform.Endpoint(endpoint_id)
    
    # Create sample test data
    test_instance = {
        "date": "2025-03-01",
        "location": "Iowa",
        "crop_type": "corn",
        "field_size": 4.5,
        "rainfall": 700,
        "temperature": 23
    }
    
    # Make prediction
    prediction = endpoint.predict([test_instance])
    print("Tabular Model Prediction:")
    print(f"Predicted Yield: {prediction}")

# Replace with your tabular endpoint ID
tabular_endpoint_id = "YOUR_TABULAR_ENDPOINT_ID"

# Uncomment to test
# test_tabular_model(tabular_endpoint_id)

## 5. Get Model Metrics

In [ ]:
def get_model_metrics(model_id):
    """Get evaluation metrics for a model"""
    model = aiplatform.Model(model_id)
    
    print(f"\nMetrics for model: {model.display_name}")
    for metrics in model.list_model_evaluations():
        print(f"Evaluation ID: {metrics.name}")
        print(f"Metrics: {metrics.metrics}")
        
# Replace with your model IDs
vision_model_id = "YOUR_VISION_MODEL_ID"
tabular_model_id = "YOUR_TABULAR_MODEL_ID"

# Uncomment to get metrics
# print("Vision Model Metrics:")
# get_model_metrics(vision_model_id)
# print("\nTabular Model Metrics:")
# get_model_metrics(tabular_model_id)